# Export Predictions to CSV (for R)

In [1]:
import os
import matplotlib.pyplot as plt
import yaml
#from ct_classifier/ct_classifier.train import create_dataloader, load_model       # NOTE: since we're using these functions across files, it could make sense to put them in e.g. a "util.py" script.
import torch
import pandas as pd
import numpy as np
from torch import softmax as softmax

/home/magali/miniconda3/envs/cv4e_env_new/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#os.getcwd()
os.chdir('/home/magali/ct_classifier/ct_classifier')

In [3]:
from train import load_model
from train import create_dataloader

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: You are trying to log string value as a metric. This is not recommended.
COMET INFO: Experiment is live on comet.com https://www.comet.com/magalifr/general/cb23bd590157453991e54245faf34a76



## Parameters

In [4]:
config = '/home/magali/ct_classifier/configs/exp_resnet18_sex.yaml'
split = 'val'

## Load Data

In [5]:
# load config
print(f'Using config "{config}"')
cfg = yaml.safe_load(open(config, 'r'))


# setup entities
dl_val = create_dataloader(cfg, split='val')

# load model
model = load_model(cfg)[0] #takes first model, refer to bets model here

Using config "/home/magali/ct_classifier/configs/exp_resnet18_sex.yaml"
val number of images 1540 labels 1540 images covered 1540
Starting new model


## Visualize

This is up to you to figure out now. :)

In [6]:
print(type(model))
#print(model)
#print(prediction)

<class 'model.CustomResNet18'>


In [7]:
device = cfg['device']
model.to(device)

model.eval()

CustomResNet18(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [8]:
all_pred_labels = []
all_labels = []
all_img_path = []

def tuple_to_list(obj):
    if isinstance(obj, tuple):
        return [tuple_to_list(item) for item in obj]
    return obj

with torch.no_grad():               # don't calculate intermediate gradient steps: we don't need them, so this saves memory and is faster
        for idx, (data, labels, img_path) in enumerate(dl_val):

            # put data and labels on device
            data, labels = data.to(device), labels.to(device)

            print(data.shape)
            #print(model.classifier.device)
            print(data.device)
            print(device)
            # forward pass
            prediction = model(data)
            #print(prediction[0])
            #print(img_path[0])

            # If you want to track labels and paths as well, you can continue to do that
            pred_label = prediction
            all_pred_labels = all_pred_labels + pred_label.cpu().tolist()
            print(all_pred_labels)
            all_labels = all_labels + labels.tolist()
            print(all_labels)

            # Convert tuple to numpy array
            #img_path_arr = np.array(img_path)
            #img_path_lst = tuple_to_list(img_path)

            #all_img_path = all_img_path + img_path_arr.tolist()

            # Flatten the img_path tuple
            img_path_lst = tuple_to_list(img_path)
            all_img_path.extend(img_path_lst)

            # Print sizes for debugging
            print(f"Iteration {idx}: all_pred_labels size: {len(all_pred_labels)}, all_labels size: {len(all_labels)}, all_img_path size: {len(all_img_path)}")

            
        # After the loop, print or return the summed predictions
        print(all_pred_labels, all_labels, all_img_path)




torch.Size([128, 3, 224, 224])
cuda:0
cuda
[[0.35323366522789, 1.2205686569213867], [0.20658710598945618, 0.2783888280391693], [0.30469489097595215, 0.6716307401657104], [0.410251647233963, 1.2426704168319702], [0.36477673053741455, 1.0473735332489014], [0.15801644325256348, 0.891956627368927], [0.6141707897186279, 0.7398741841316223], [0.35750120878219604, 1.0698295831680298], [0.65589839220047, 1.3122594356536865], [0.6898089051246643, 0.45281460881233215], [0.7820742726325989, 0.8144335746765137], [0.3157689869403839, 0.42320936918258667], [0.2452651858329773, 0.4355561137199402], [0.5131203532218933, 0.8028385639190674], [0.5904191136360168, 0.8869165778160095], [0.7731195688247681, 0.7207476496696472], [0.5432087779045105, 0.9621135592460632], [0.9300851225852966, 1.016302227973938], [0.7800332307815552, 1.2166481018066406], [0.6603574156761169, 0.18682076036930084], [0.6030562520027161, 0.7173197269439697], [0.6412009596824646, 0.5677132606506348], [0.40810656547546387, 0.6423570

In [9]:
print(all_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 

Create dataframes with predictions, labels and image id

In [10]:
label_df= pd.DataFrame(all_labels, columns=['Labels'])
pred_df = pd.DataFrame(all_pred_labels, columns=['pred_female', 'pred_male'])
img_id_df = pd.DataFrame(all_img_path, columns=['Image path'])


In [11]:
print(pred_df)

      pred_female  pred_male
0        0.353234   1.220569
1        0.206587   0.278389
2        0.304695   0.671631
3        0.410252   1.242670
4        0.364777   1.047374
...           ...        ...
1535     0.683889   0.240454
1536     0.753208   0.301511
1537     0.364439   0.783581
1538     0.368737   0.681050
1539     0.507086   0.822741

[1540 rows x 2 columns]


In [12]:
print(img_id_df)

                                             Image path
0     /home/magali/CV4Ecology-summer-school/Prototyp...
1     /home/magali/CV4Ecology-summer-school/Prototyp...
2     /home/magali/CV4Ecology-summer-school/Prototyp...
3     /home/magali/CV4Ecology-summer-school/Prototyp...
4     /home/magali/CV4Ecology-summer-school/Prototyp...
...                                                 ...
1535  /home/magali/CV4Ecology-summer-school/Prototyp...
1536  /home/magali/CV4Ecology-summer-school/Prototyp...
1537  /home/magali/CV4Ecology-summer-school/Prototyp...
1538  /home/magali/CV4Ecology-summer-school/Prototyp...
1539  /home/magali/CV4Ecology-summer-school/Prototyp...

[1540 rows x 1 columns]


In [13]:
result = pd.concat([label_df, pred_df, img_id_df], axis=1, ignore_index=False)
print(result)

      Labels  pred_female  pred_male  \
0          0     0.353234   1.220569   
1          0     0.206587   0.278389   
2          0     0.304695   0.671631   
3          0     0.410252   1.242670   
4          0     0.364777   1.047374   
...      ...          ...        ...   
1535       0     0.683889   0.240454   
1536       0     0.753208   0.301511   
1537       0     0.364439   0.783581   
1538       0     0.368737   0.681050   
1539       0     0.507086   0.822741   

                                             Image path  
0     /home/magali/CV4Ecology-summer-school/Prototyp...  
1     /home/magali/CV4Ecology-summer-school/Prototyp...  
2     /home/magali/CV4Ecology-summer-school/Prototyp...  
3     /home/magali/CV4Ecology-summer-school/Prototyp...  
4     /home/magali/CV4Ecology-summer-school/Prototyp...  
...                                                 ...  
1535  /home/magali/CV4Ecology-summer-school/Prototyp...  
1536  /home/magali/CV4Ecology-summer-school/Prototyp...

convert predictions to with softmax

In [14]:
# Define a function to apply torch.softmax to a list of scores
#def apply_softmax(scores):
#    tensor_scores = torch.tensor(scores)
#    softmax_scores = torch.softmax(tensor_scores, dim=0).tolist()  # Using dim=0 for 1D tensor
#    return softmax_scores

# Apply the function to the 'prediction' column
#result['pred_female_softmax'] = result['pred_female'].apply(torch.softmax)
#result['pred_male_softmax'] = result['pred_male'].apply(apply_softmax)

# Convert DataFrame columns to a tensor
tensor_predictions = torch.tensor(result[['pred_female', 'pred_male']].values)

# Compute softmax along the last dimension (across columns for each row)
softmax_tensor = torch.softmax(tensor_predictions, dim=1)

# Convert the tensor back to a DataFrame
df_softmax = pd.DataFrame(softmax_tensor.numpy(), columns=['pred_female_softmax', 'pred_male_softmax'])

# Concatenate the two DataFrames along axis=1
result = pd.concat([result, df_softmax], axis=1)

print(result)


      Labels  pred_female  pred_male  \
0          0     0.353234   1.220569   
1          0     0.206587   0.278389   
2          0     0.304695   0.671631   
3          0     0.410252   1.242670   
4          0     0.364777   1.047374   
...      ...          ...        ...   
1535       0     0.683889   0.240454   
1536       0     0.753208   0.301511   
1537       0     0.364439   0.783581   
1538       0     0.368737   0.681050   
1539       0     0.507086   0.822741   

                                             Image path  pred_female_softmax  \
0     /home/magali/CV4Ecology-summer-school/Prototyp...             0.295809   
1     /home/magali/CV4Ecology-summer-school/Prototyp...             0.482057   
2     /home/magali/CV4Ecology-summer-school/Prototyp...             0.409282   
3     /home/magali/CV4Ecology-summer-school/Prototyp...             0.303134   
4     /home/magali/CV4Ecology-summer-school/Prototyp...             0.335682   
...                                    

In [15]:
print(result)

      Labels  pred_female  pred_male  \
0          0     0.353234   1.220569   
1          0     0.206587   0.278389   
2          0     0.304695   0.671631   
3          0     0.410252   1.242670   
4          0     0.364777   1.047374   
...      ...          ...        ...   
1535       0     0.683889   0.240454   
1536       0     0.753208   0.301511   
1537       0     0.364439   0.783581   
1538       0     0.368737   0.681050   
1539       0     0.507086   0.822741   

                                             Image path  pred_female_softmax  \
0     /home/magali/CV4Ecology-summer-school/Prototyp...             0.295809   
1     /home/magali/CV4Ecology-summer-school/Prototyp...             0.482057   
2     /home/magali/CV4Ecology-summer-school/Prototyp...             0.409282   
3     /home/magali/CV4Ecology-summer-school/Prototyp...             0.303134   
4     /home/magali/CV4Ecology-summer-school/Prototyp...             0.335682   
...                                    

In [16]:
result.to_csv('../predictions/predictions_sex_basic.csv', index=True)
